In [1]:
# Import and download packages
import pandas as pd
import glob
import nltk
from nltk.stem import WordNetLemmatizer
import codecs
import json
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/davidguo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davidguo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Read CSV into DataFrame
df = pd.read_csv('./word_list/middle_school_vocab.csv')

In [3]:
# Clean up DataFrame
df['in_text'] = 0
df['word'] = df['word'].str.lower()

In [4]:
# Read text files
hp_files = glob.glob("hp_text/*[1-7].txt")

# Load BrE to AmE dictionary
b2a_dict = json.load(open('b2a.json'))

In [5]:
word_list = []
for file in hp_files:
    with codecs.open(file, 'r', encoding='utf-8',
                 errors='ignore') as file_object:
        content = file_object.read()
        words_in_content = nltk.word_tokenize(content)
        for word in words_in_content:
            ame_word = b2a_dict.get(word)
            lemmatizer = WordNetLemmatizer()
            if ame_word:                
                word_list.append(lemmatizer.lemmatize(ame_word).lower())
            else:
                word_list.append(lemmatizer.lemmatize(word).lower())
deduped_word_list = list(set(word_list))

In [6]:
i_max = df.shape[0]
i = 0
j = 0

while i < i_max:
    lemmatizer = WordNetLemmatizer()
    loc_word = df['word'].loc[i]
    lem_word = lemmatizer.lemmatize(loc_word).lower()
    ame_word = b2a_dict.get(lem_word)
    if lem_word in deduped_word_list:
        df.at[i, 'in_text'] = 1
        j += 1
    elif ame_word in deduped_word_list:
        df.at[i, 'in_text'] = 1
        j += 1
    i += 1

In [7]:
output = str(round(j / i_max * 100, 2)) + "%"
print(output)

79.46%


In [8]:
df_not_in_text = df.query('in_text == 0')
df_not_in_text.to_csv('not_in_text.csv', index=False)
